docoding过程的采样策略

In [ ]:
docoding过程的采样策略主要包括以下几种，每种策略的原理和适用场景如下：

1. 贪心采样（Greedy Sampling）：
    - 原理：每一步都选择概率最大的token作为下一个输出。
    - 优点：生成结果确定、可复现，速度快。
    - 缺点：容易陷入局部最优，生成文本缺乏多样性，可能出现重复或不自然的表达。
    - 适用场景：对确定性和可控性要求高的任务，如摘要、翻译等。

2. 随机采样（Random Sampling）：
    - 原理：根据模型输出的概率分布，直接对所有token进行随机采样，选择下一个token。
    - 优点：生成文本多样性高，能产生新颖内容。
    - 缺点：可能采样到概率很低、不合理的token，导致文本不连贯或语义错误。
    - 适用场景：需要创造性或多样性输出的场景，如对话生成、故事创作等。

3. Top-k采样：
    - 原理：在每一步只保留概率最高的k个token（即top-k），然后在这k个token中按概率归一化后随机采样。
    - 优点：限制了候选token范围，减少生成低概率、不合理token的风险，同时保留一定多样性。
    - 缺点：k值选择不当会影响生成质量，k太小接近贪心采样，k太大接近随机采样。
    - 适用场景：需要在合理性和多样性之间平衡的生成任务。

4. Top-p采样（Nucleus Sampling）：
    - 原理：动态选择累计概率达到p（如0.9）的最小token集合，然后在该集合内按概率归一化后随机采样。
    - 优点：自适应调整候选集大小，能更好地平衡多样性和合理性，避免固定k带来的问题。
    - 缺点：p值设置不当会影响生成效果，p太小限制多样性，p太大降低合理性。
    - 适用场景：广泛用于对话、文本生成等多样性要求较高的任务。

5. 温度采样（Temperature Sampling）：
    - 原理：通过温度参数T调整概率分布，softmax(logits/T)。T>1使分布更平坦，T<1使分布更尖锐。
    - 优点：灵活控制生成文本的随机性和确定性。高温度增加多样性，低温度增强确定性。
    - 缺点：极端温度值会导致文本质量下降（T过高随机性过大，T过低接近贪心）。
    - 适用场景：可与其他采样策略结合，微调生成风格和多样性。

实际应用中，常常将Top-k/Top-p采样与温度采样结合使用，以获得更优的生成效果。选择合适的采样策略和参数，需要根据具体任务和需求进行实验和调整。

In [ ]:
### Beam Search（束搜索）

**原理**：束搜索（Beam Search）是一种在序列生成任务中常用的启发式搜索算法。其核心思想是在每个解码步骤中，保留 b 个概率最高的候选序列（b 称为束宽 beam width），而不是像贪心采样那样只保留一个最优序列。这样可以在保证生成质量的同时，兼顾一定的多样性和全局最优的可能性。

**具体流程：**

1. **初始化**：将起始符（如BOS）作为唯一的初始候选序列，概率为1。
2. **扩展候选**：对于当前所有候选序列，每个都扩展所有可能的下一个token，计算每个扩展序列的累积概率。
3. **筛选排序**：从所有扩展出来的新序列中，选取概率最高的 b 个，作为下一步的候选集合。
4. **终止条件**：当所有候选序列都生成了结束符（EOS）或达到最大长度时，停止搜索。
5. **输出结果**：通常从最终候选集合中选择累积概率最大的序列作为最终输出。

**公式表达：**

$$
T^* = \arg\max_{T \in B} P(T|x)
$$

其中 $B$ 是最终的候选集合，$x$ 是输入序列。

**优点**：
- 生成结果通常更流畅和高质量，能避免贪心采样带来的局部最优。
- 适用于机器翻译、摘要等对生成质量要求高的任务。

**缺点**：
- 计算量较大，束宽越大，计算资源消耗越多。
- 可能导致生成内容重复或缺乏多样性（束搜索偏向高概率序列，容易陷入模式化表达）。
- 不适合需要极高多样性的生成任务（如开放式对话）。

**实际应用建议**：
- 束宽 b 一般设置为3~10，过大提升有限且消耗资源。
- 可与长度惩罚、重复惩罚等机制结合，提升生成多样性和合理性。